In [1]:
# Imports and libs
import os
import os.path
import pathlib

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import io
import scipy.misc
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub

import PIL.Image as Image
import PIL.ImageColor as ImageColor
import PIL.ImageDraw as ImageDraw
import PIL.ImageFont as ImageFont

import six

from concurrent.futures import ThreadPoolExecutor

import collections

import ipywidgets as widgets
from IPython.display import display

import time

tf.debugging.set_log_device_placement(True)

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
# List of all models endorsed by the official tf object detection tutorial that were trained on the COCO 2017 training set
ALL_MODELS = {
    'EfficientDet' : 'https://tfhub.dev/tensorflow/efficientdet/d7/1',
    'Faster R-CNN' : 'https://tfhub.dev/tensorflow/faster_rcnn/inception_resnet_v2_1024x1024/1',
    'Faster R-CNN (OpenImages)' : 'https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1',
    'CenterNet' : 'https://tfhub.dev/tensorflow/centernet/resnet101v1_fpn_512x512/1'
}

In [4]:
# Load object detection lib
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import ops as utils_ops

%matplotlib inline

# Path to predefined labels for the openimages v4 training set
PATH_TO_LABELS = './tensorflow_models/research/object_detection/data/oid_v4_label_map.pbtxt'
# Path to predefined labels for the coco 2017 training set
# PATH_TO_LABELS = './tensorflow_models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

ModuleNotFoundError: No module named 'object_detection'

In [ ]:
# Select model
model_display_name = 'EfficientDet'
model_handle = ALL_MODELS[model_display_name]

print('Selected model: '+ model_display_name)
print('Model Handle at TensorFlow Hub: {}'.format(model_handle))

# Load model
hub_model = hub.load(model_handle)
print('Model loaded!')

In [ ]:
# Load images from inputs folder
IMAGES = []
VALID_PATHS = []
running = []

# Get all valid image paths
# image_type = 'colorized_artistic' # use this for brighter/more daring colors
# image_type = 'grayscale' # use this for the original grayscale images
image_type = 'colorized_stable' # use this for most stable results
file_path = 'inputs/' + image_type + '/'
for image_path in os.listdir(file_path):
    if (image_path.endswith('.jpg')):
        # Collect image as valid path
        VALID_PATHS.append(image_path)
        
progress_load = widgets.IntProgress(min=0, max=len(VALID_PATHS), description='Loaded: ') # instantiate the bar
load_label = widgets.Label(value='0 / ' + str(len(VALID_PATHS)))
display(widgets.HBox([progress_load, load_label])) # display the bar

# Load given image into numpy array using matplotlib imload
def load_valid_image(path):
    img = mpimg.imread(file_path + path, format='jpg')
    # Reshape image array to fit specifications required by tensorflow model
    (rows, columns, channels) = img.shape
    IMAGES.append({
        'image': img.reshape((1, rows, columns, channels)),
        'filename': path,
        'detection': {}
    })
    progress_load.value += 1
    load_label.value = str(progress_load.value) + ' / ' + str(len(VALID_PATHS))

# Parallelize execution with threadpool
def run_with_Threadpool(paths):
    with ThreadPoolExecutor(max_workers=(os.cpu_count() * 5)) as executor:
        # Schedule task for each image
        running = [executor.submit(
            lambda: load_valid_image(path)
        ) for path in paths]

"""
    Since we are working with a lot of IO operations, we use Threadpool since it is much faster for IO than using Processes:
    
    Loading 20 images:
        Runtime with Processes: ~20 seconds
        Runtime with Threadpool: ~3 seconds
"""
# Load all valid images
# start_time = time.time()
# run_with_Threadpool(VALID_PATHS)
# print("--- Runtime with Threadpool: %s seconds ---" % (time.time() - start_time))

for path in VALID_PATHS:
    load_valid_image(path)
    
print('Done!')

In [ ]:
# Draw first image
@widgets.interact(i=(0, len(IMAGES)))
def show_image_at(i=0):
    plt.figure(figsize=(18,26))
    plt.imshow(IMAGES[i]['image'][0])
    plt.show()

In [ ]:
# running inference
# different object detection models have additional results
# all of them are explained in the documentation
progress_inference = widgets.IntProgress(min=0, max=len(IMAGES), description='Inferred: ')
inference_label = widgets.Label(value='0 / ' + str(len(IMAGES)))
display(widgets.HBox([progress_inference, inference_label]))

start_time = time.time()
for image in IMAGES:
    # Run object detection and save results
    detected = hub_model(image['image'])
    image['detection'] = {key:value.numpy() for key,value in detected.items()}
    
    progress_inference.value += 1
    inference_label.value = str(progress_inference.value) + ' / ' + str(len(IMAGES))
    
runtime = ((time.time() - start_time) / 60)
print("--- Runtime: " + f'{runtime:.2f}' + " minutes ---")

In [ ]:
# Copy image to draw detected objects
IMAGES_WITH_DETECTION = []
for image in IMAGES:
    IMAGES_WITH_DETECTION.append({
        'image': image['image'].copy(),
        'filename': image['filename'],
        'detection': image['detection']
    })

In [ ]:
# Use shortened version of function from tensorflow library, to add font-size support
STANDARD_COLORS = [
    'AliceBlue', 'Chartreuse', 'Aqua', 'Aquamarine', 'Azure', 'Beige', 'Bisque',
    'BlanchedAlmond', 'BlueViolet', 'BurlyWood', 'CadetBlue', 'AntiqueWhite',
    'Chocolate', 'Coral', 'CornflowerBlue', 'Cornsilk', 'Crimson', 'Cyan',
    'DarkCyan', 'DarkGoldenRod', 'DarkGrey', 'DarkKhaki', 'DarkOrange',
    'DarkOrchid', 'DarkSalmon', 'DarkSeaGreen', 'DarkTurquoise', 'DarkViolet',
    'DeepPink', 'DeepSkyBlue', 'DodgerBlue', 'FireBrick', 'FloralWhite',
    'ForestGreen', 'Fuchsia', 'Gainsboro', 'GhostWhite', 'Gold', 'GoldenRod',
    'Salmon', 'Tan', 'HoneyDew', 'HotPink', 'IndianRed', 'Ivory', 'Khaki',
    'Lavender', 'LavenderBlush', 'LawnGreen', 'LemonChiffon', 'LightBlue',
    'LightCoral', 'LightCyan', 'LightGoldenRodYellow', 'LightGray', 'LightGrey',
    'LightGreen', 'LightPink', 'LightSalmon', 'LightSeaGreen', 'LightSkyBlue',
    'LightSlateGray', 'LightSlateGrey', 'LightSteelBlue', 'LightYellow', 'Lime',
    'LimeGreen', 'Linen', 'Magenta', 'MediumAquaMarine', 'MediumOrchid',
    'MediumPurple', 'MediumSeaGreen', 'MediumSlateBlue', 'MediumSpringGreen',
    'MediumTurquoise', 'MediumVioletRed', 'MintCream', 'MistyRose', 'Moccasin',
    'NavajoWhite', 'OldLace', 'Olive', 'OliveDrab', 'Orange', 'OrangeRed',
    'Orchid', 'PaleGoldenRod', 'PaleGreen', 'PaleTurquoise', 'PaleVioletRed',
    'PapayaWhip', 'PeachPuff', 'Peru', 'Pink', 'Plum', 'PowderBlue', 'Purple',
    'Red', 'RosyBrown', 'RoyalBlue', 'SaddleBrown', 'Green', 'SandyBrown',
    'SeaGreen', 'SeaShell', 'Sienna', 'Silver', 'SkyBlue', 'SlateBlue',
    'SlateGray', 'SlateGrey', 'Snow', 'SpringGreen', 'SteelBlue', 'GreenYellow',
    'Teal', 'Thistle', 'Tomato', 'Turquoise', 'Violet', 'Wheat', 'White',
    'WhiteSmoke', 'Yellow', 'YellowGreen'
]

def visualize_boxes_and_labels_on_image_array(
    image,
    boxes,
    classes,
    scores,
    category_index,
    use_normalized_coordinates=False,
    max_boxes_to_draw=20,
    min_score_thresh=.5,
    agnostic_mode=False,
    line_thickness=4,
    font_size=24,
    mask_alpha=.4,
    groundtruth_box_visualization_color='black'):

  # Create a display string (and color) for every box location, group any boxes
  # that correspond to the same location.
  box_to_display_str_map = collections.defaultdict(list)
  box_to_color_map = collections.defaultdict(str)
  box_to_track_ids_map = {}
  if not max_boxes_to_draw:
    max_boxes_to_draw = boxes.shape[0]
  for i in range(boxes.shape[0]):
    if max_boxes_to_draw == len(box_to_color_map):
      break
    if scores is None or scores[i] > min_score_thresh:
      box = tuple(boxes[i].tolist())
      if scores is None:
        box_to_color_map[box] = groundtruth_box_visualization_color
      else:
        display_str = ''
        if not agnostic_mode:
            if classes[i] in six.viewkeys(category_index):
              class_name = category_index[classes[i]]['name']
            else:
              class_name = 'N/A'
            display_str = str(class_name)
        if not display_str:
            display_str = '{}%'.format(round(100*scores[i]))
        else:
            display_str = '{}: {}%'.format(display_str, round(100*scores[i]))
        box_to_display_str_map[box].append(display_str)
        if agnostic_mode:
          box_to_color_map[box] = 'DarkOrange'
        else:
          box_to_color_map[box] = STANDARD_COLORS[
              classes[i] % len(STANDARD_COLORS)]

  # Draw all boxes onto image.
  for box, color in box_to_color_map.items():
    ymin, xmin, ymax, xmax = box
    draw_bounding_box_on_image_array(
        image,
        ymin,
        xmin,
        ymax,
        xmax,
        color=color,
        thickness=line_thickness,
        font_size=font_size,
        display_str_list=box_to_display_str_map[box],
        use_normalized_coordinates=use_normalized_coordinates)

  return image

def draw_bounding_box_on_image_array(
    image,
    ymin,
    xmin,
    ymax,
    xmax,
    color='red',
    thickness=4,
    font_size=24,
    display_str_list=(),
    use_normalized_coordinates=True):

  image_pil = Image.fromarray(np.uint8(image)).convert('RGB')
  draw_bounding_box_on_image(image_pil, ymin, xmin, ymax, xmax, color,
                             thickness, font_size, display_str_list,
                             use_normalized_coordinates)
  np.copyto(image, np.array(image_pil))

def draw_bounding_box_on_image(
    image,
    ymin,
    xmin,
    ymax,
    xmax,
    color='red',
    thickness=4,
    font_size=24,
    display_str_list=(),
    use_normalized_coordinates=True):

  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  if use_normalized_coordinates:
    (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                  ymin * im_height, ymax * im_height)
  else:
    (left, right, top, bottom) = (xmin, xmax, ymin, ymax)
  if thickness > 0:
    draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
               (left, top)],
              width=thickness,
              fill=color)
  try:
    font = ImageFont.truetype('arial.ttf', font_size)
  except IOError:
    font = ImageFont.load_default()

  # If the total height of the display strings added to the top of the bounding
  # box exceeds the top of the image, stack the strings below the bounding box
  # instead of above.
  display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
  # Each display_str has a top and bottom margin of 0.05x.
  total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

  if top > total_display_str_height:
    text_bottom = top
  else:
    text_bottom = bottom + total_display_str_height
  # Reverse list and print from bottom to top.
  for display_str in display_str_list[::-1]:
    text_width, text_height = font.getsize(display_str)
    margin = np.ceil(0.05 * text_height)
    draw.rectangle(
        [(left, text_bottom - text_height - 2 * margin), (left + text_width,
                                                          text_bottom)],
        fill=color)
    draw.text(
        (left + margin, text_bottom - text_height - margin),
        display_str,
        fill='black',
        font=font)
    text_bottom -= text_height - 2 * margin

In [ ]:
# Draw detections on copied image
for image in IMAGES_WITH_DETECTION:
    visualize_boxes_and_labels_on_image_array(
          image['image'][0],
          image['detection']['detection_boxes'][0],
          (image['detection']['detection_classes'][0]).astype(int),
          image['detection']['detection_scores'][0],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          # Change this if confidence score should be higher or lower
          min_score_thresh=.50,
          line_thickness=6,
          font_size=42,
          agnostic_mode=False
    )

In [ ]:
# Display images with detection
@widgets.interact(i=(0, len(IMAGES_WITH_DETECTION)))
def show_image_at(i=0):
    plt.figure(figsize=(24,32))
    plt.imshow(IMAGES_WITH_DETECTION[i]['image'][0])
    plt.show()

In [ ]:
# Add output sub-directory named after the model that was used (if it doesn't already exist)
output_dir = 'outputs/' + model_display_name + '/' + image_type + '/'
pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True)

progress_save = widgets.IntProgress(min=0, max=len(IMAGES_WITH_DETECTION), description='Saved: ')
save_label = widgets.Label(value='0 / ' + str(len(IMAGES_WITH_DETECTION)))
display(widgets.HBox([progress_save, save_label]))

# Save images
for image in IMAGES_WITH_DETECTION:
    plt.imsave(output_dir + image['filename'], image['image'][0], format='jpg')
    
    progress_save.value += 1
    save_label.value = str(progress_save.value) + ' / ' + str(len(IMAGES_WITH_DETECTION))